In [ ]:
%matplotlib inline

import cv2
from PIL import Image
import matplotlib.pyplot as plt

import numpy as np

from pathlib import Path
from glob import glob

from pprint import pprint
from tqdm.auto import tqdm

import utility

CACHE_DIR = Path('./caches')
utility.create_directory(CACHE_DIR)

## Preprocess

In [ ]:
as_deeplearning_input = True
deeplearning_input_shape = (800, 600)
files = glob('datasets/20230204/*sample*.jpg')

In [ ]:
for f_name in tqdm(files):
    im = cv2.imread(f_name)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # Gaussian Blur
    im_smoothed = cv2.GaussianBlur(im, (9, 9), 0)

    # Average Blur
    kernel = np.ones((5, 5), np.float32) / 25
    im_smoothed = cv2.filter2D(im_smoothed, -1, kernel)
    im_smoothed = cv2.medianBlur(im_smoothed, 25)

    # HSV get luminance info
    hsv_im = cv2.cvtColor(im_smoothed, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_im)
    im_gray = v

    # Otsu's thresholding
    _, im_thres = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Find Contours bounding box
    contours, hierarchy = cv2.findContours(im_thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)

    im = im[y:y+h, x:x+w]
    if as_deeplearning_input:
        im = utility.mutaSCAN_image_resize(im, *deeplearning_input_shape)
    img_saving = Image.fromarray(im)

    save_name = CACHE_DIR / Path('test_samples') / Path(f_name).stem
    save_name =  Path(save_name).as_posix() + '_processed.png'
    img_saving.save(save_name)

## Debug

In [ ]:
f_name = 'datasets/20230128_190030_232300_sample0.jpg'
figsize=(10, 10)

In [ ]:
im = cv2.imread(f_name)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

plt.figure(figsize=figsize)
plt.imshow(im)

In [ ]:
# Gaussian Blur
im_smoothed = cv2.GaussianBlur(im, (9, 9), 0)

# Average Blur
kernel = np.ones((5, 5), np.float32) / 25
im_smoothed = cv2.filter2D(im_smoothed, -1, kernel)

im_smoothed = cv2.medianBlur(im_smoothed, 25)

plt.figure(figsize=figsize)
plt.imshow(im_smoothed)

In [ ]:
hsv_im = cv2.cvtColor(im_smoothed, cv2.COLOR_RGB2HSV)
h, s, v = cv2.split(hsv_im)
im_gray = v
plt.figure(figsize=figsize)
plt.imshow(v, cmap='gray')

In [ ]:
_, im_thres = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

plt.figure(figsize=figsize)
plt.imshow(im_thres, cmap='gray')

In [ ]:
contours, hierarchy = cv2.findContours(im_thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
c = max(contours, key=cv2.contourArea)
masked = cv2.drawContours(im, [c], -1, (255, 0, 0), 5)
x, y, w, h = cv2.boundingRect(c)
cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 5)

plt.figure(figsize=figsize)
plt.imshow(im)